In [5]:
import tensorflow as tf

import os 

from tensorflow.keras.layers import Input

In [7]:
def load(image_file):
    #decode into uint8 tensor
    image = tf.io.read_file(image_file)
    image = tf.io.decode_jpeg(image)

    #split into photo + painting 
    w = tf.shape(image)[1]
    w = w // 2
    input_image = image[:,w:,:]
    real_image = image[:,:w,:]

    #convert both images to float32 tensors 
    input_image = tf.cast(input_image, tf.float32)
    real_image = tf.cast(real_image, tf.float32)

    return input_image, real_image 



In [ ]:
def define_discriminator():

    initialzer = tf.random_normal_initializer(0.0, 0.05)

    inp = Input(shape=[256, 256, 3], name='input_image')
    target_img = Input(shape=[256, 256, 3], name='target_image')

    merged = Concatenate()([src_img,target_img])